# Import dependencies

In [2]:
import gymnasium as gym
from gymnasium import Env
from gymnasium.spaces import Discrete, Box

import numpy as np
import random
import os

from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

2024-05-01 23:42:15.534918: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 23:42:16.682544: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
log_path = os.path.join('Training', 'House_Logs') 
training_log_path = os.path.join(log_path, 'SAC_1')

!tensorboard --logdir={training_log_path}

2024-05-01 23:42:36.756271: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 23:42:37.773542: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.16.2 at http://localhost:6006/ (Press CTRL+C to quit)


# Building an enviroment

The ResourceManagementEnv class implements the resource management environment. 

Resources are represented as a vector of numeric values that can range from 0 to a predefined maximum value. The goal of the agent is to allocate the resources to achieve the desired resource configuration (goal_resources).

In [176]:
# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = Discrete(3)  # Increase(1), Decrease(-1) or Keep(0) actions

#         # Define the observation space
#         self.observation_space = Box(low=0, high=100, shape=(1,))  # Energy level ranging from 0 to 100

#         # Define the initial energy level
#         self.energy_level = random.randint(50, 80)

#         # Define the resource manager properties
#         self.energy_capacity = 100  # Maximum energy capacity
#         self.energy_threshold = 60  # Energy threshold for energy-saving mode
#         self.energy_saving_factor = 0.5  # Factor by which energy is saved when in energy-saving mode
#         self.energy_usage = 5  # Energy used per time step

#         # Define the number of time steps for the resource manager 
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#     def step(self, action):
#         # Apply the action to change the energy level
#         if action == 0:  # Decrease energy
#             self.energy_level -= self.energy_usage
#         elif action == 1:  # Keep energy level (no change)
#             pass
#         elif action == 2:  # Increase energy
#             self.energy_level += self.energy_usage

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(self.energy_capacity, self.energy_level))

#         # Calculate reward based on energy-saving state
#         if self.energy_level >= self.energy_threshold:
#             reward = 1
#         else:
#             reward = -1

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         return self.energy_level, reward, done, truncated, info

#     def reset(self, seed=None):
#         # Reset the environment to the initial state
#         self.energy_level = np.array([random.randint(50, 80)]).astype(float)
#         self.current_step = 0

#         return self.energy_level, {}

#     def render(self):
#         # Implement visualization (optional)
#         pass

In [177]:
# import gym
# from gym import spaces
# import numpy as np
# import random

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.num_actions = 11  # Number of discrete actions
#         self.action_space = spaces.Discrete(self.num_actions)  # Discrete action space

#         # Define the observation space
#         self.observation_space = spaces.Box(low=np.array([0, 18]), high=np.array([100, 32]), shape=(2,), dtype=np.float32)  # Energy level and room temperature

#         # Define the initial state
#         self.energy_level = random.randint(50, 80)
#         self.base_temperature = 26  # Base room temperature
#         self.room_temperature = self.generate_initial_temperature()  # Initial room temperature

#         # Define environment parameters
#         self.energy_capacity = 200
#         self.energy_usage_low = 2
#         self.energy_usage_high = 8
#         self.heater_efficiency = 0.5  # Efficiency of the heater pump
#         self.time_steps = 100
#         self.current_step = 0

#         # Define temperature dynamics parameters
#         self.temperature_amplitude = 5  # Amplitude of temperature variation
#         self.temperature_period = 24  # Period of temperature variation (hours)
#         self.temperature_phase = random.uniform(0, 2*np.pi)  # Random phase shift

#     def step(self, action):
#         # Convert discrete action to continuous value
#         heater_power = (action / (self.num_actions - 1)) * 2 - 1  # Scale action to range [-1, 1]

#         # Apply action to regulate heater pump power
#         energy_usage = max(0, heater_power) * self.heater_efficiency

#         # Update room temperature based on heater pump power and energy usage
#         temperature_variation = self.calculate_temperature_variation()
#         self.room_temperature = self.base_temperature + temperature_variation

#         print("Room temperature:", self.room_temperature)  # Add this line for debugging

#         # Update energy level based on heater pump power and energy usage
#         energy_usage += random.randint(self.energy_usage_low, self.energy_usage_high)
#         self.energy_level -= energy_usage

#         # Clip energy level within the valid range
#         self.energy_level = np.clip(self.energy_level, 0, self.energy_capacity)

#         # Calculate reward based on room temperature proximity to base temperature
#         temperature_variation = self.calculate_temperature_variation()
#         max_temp_diff = self.temperature_amplitude  # Update max_temp_diff dynamically
#         reward = (1 - abs(temperature_variation) / max_temp_diff)  # Penalize for deviation from base temperature

#         print("Reward:", reward)  # Add this line for debugging

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Additional info can be an empty dictionary
#         info = {}

#         # Return observation as a numpy array
#         if isinstance(self.energy_level, np.ndarray) and isinstance(self.room_temperature, np.ndarray):
#             observation = np.array([self.energy_level[0], self.room_temperature[0]], dtype=np.float32)
#         else:
#             observation = np.array([self.energy_level, self.room_temperature], dtype=np.float32)

#         return observation, reward, done, info


#     def reset(self):
#     # Reset the environment to the initial state
#         self.energy_level = random.randint(50, 80)
#         self.room_temperature = self.generate_initial_temperature()
#         print("Initial room temperature:", self.room_temperature)  # Add this line for debugging
#         self.current_step = 0

#         return np.array([self.energy_level, self.room_temperature])


#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass

#     def generate_initial_temperature(self):
#         # Generate an initial room temperature with slight variation
#         return self.base_temperature + random.uniform(-1, 1)

#     def calculate_temperature_variation(self):
#         # Calculate the sinusoidal variation of room temperature over time
#         time = self.current_step / self.temperature_period * 2 * np.pi + self.temperature_phase
#         return self.temperature_amplitude * np.sin(time)


# Solar

In [178]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(3)  # Increase(1), Decrease(-1) or Keep(0) actions

#         # Define the observation space
#         self.observation_space = spaces.Box(low=0, high=100, shape=(1,), dtype=np.float32)  # Energy level ranging from 0 to 100

#         # Define the initial energy level
#         self.energy_level = random.randint(50, 80)

#         # Define the resource manager properties
#         self.energy_capacity = 100  # Maximum energy capacity
#         self.energy_threshold = 60  # Energy threshold for energy-saving mode
#         self.energy_saving_factor = 0.5  # Factor by which energy is saved when in energy-saving mode
#         self.energy_usage = 5  # Energy used per time step

#         # Define the number of time steps for the resource manager 
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a sinusoidal pattern
#         max_solar_radiation = 1000  # Maximum solar radiation in watts per square meter
#         amplitude = max_solar_radiation / 2
#         mean_solar_radiation = max_solar_radiation / 2
#         solar_radiation = amplitude * np.sin(np.pi * (time_of_day / 24)) + mean_solar_radiation
#         return solar_radiation

#     def step(self, action):
#         # Apply the action to change the energy level
#         if action == 0:  # Decrease energy
#             self.energy_level -= self.energy_usage
#         elif action == 1:  # Keep energy level (no change)
#             pass
#         elif action == 2:  # Increase energy
#             # Calculate time-dependent solar radiation
#             solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)

#             # Calculate energy produced by solar panels
#             solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation
#             self.energy_level += solar_energy

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(self.energy_capacity, self.energy_level))

#         # Calculate reward based on energy-saving state
#         if self.energy_level >= self.energy_threshold:
#             reward = 1
#         else:
#             reward = -1

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         return np.array([self.energy_level], dtype=np.float32), reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.energy_level = random.randint(50, 80)
#         self.current_step = 0

#         return np.array([self.energy_level], dtype=np.float32)

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


# Solar + Heat

In [179]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(3)  # Increase(1), Decrease(-1) or Keep(0) actions

#         # Define the observation space
#         self.observation_space = spaces.Box(low=0, high=100, shape=(1,), dtype=np.float32)  # Energy level ranging from 0 to 100

#         # Define the initial energy level
#         self.energy_level = random.randint(50, 80)

#         # Define the resource manager properties
#         self.energy_capacity = 100  # Maximum energy capacity
#         self.energy_threshold = 60  # Energy threshold for energy-saving mode
#         self.energy_saving_factor = 0.5  # Factor by which energy is saved when in energy-saving mode
#         self.energy_usage = 5  # Energy used per time step

#         # Define the number of time steps for the resource manager 
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Add heat pump parameters
#         self.heat_pump_energy_consumption = 2  # Energy consumed by the heat pump per time step

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a sinusoidal pattern
#         max_solar_radiation = 1000  # Maximum solar radiation in watts per square meter
#         amplitude = max_solar_radiation / 2
#         mean_solar_radiation = max_solar_radiation / 2
#         solar_radiation = amplitude * np.sin(np.pi * (time_of_day / 24)) + mean_solar_radiation
#         return solar_radiation

#     def step(self, action):
#         # Apply the action to change the energy level
#         if action == 0:  # Decrease energy
#             self.energy_level -= self.energy_usage
#         elif action == 1:  # Keep energy level (no change)
#             pass
#         elif action == 2:  # Increase energy
#             # Calculate time-dependent solar radiation
#             solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)

#             # Calculate energy produced by solar panels
#             solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#             # Subtract energy consumed by the heat pump
#             total_energy_consumption = self.heat_pump_energy_consumption
#             total_energy_consumption += self.energy_usage  # Include base energy usage
#             total_energy_consumption += self.energy_usage * action  # Include additional energy usage from action
#             total_energy_consumption = min(total_energy_consumption, self.energy_level)  # Limit consumption to available energy

#             self.energy_level += solar_energy - total_energy_consumption

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(self.energy_capacity, self.energy_level))

#         # Calculate reward based on energy-saving state
#         if self.energy_level >= self.energy_threshold:
#             reward = 1
#         else:
#             reward = -1

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         return np.array([self.energy_level], dtype=np.float32), reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.energy_level = random.randint(50, 80)
#         self.current_step = 0

#         return np.array([self.energy_level], dtype=np.float32)

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


# Tried to balance it

In [180]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(3)  # Increase(1), Decrease(-1) or Keep(0) actions

#         # Define the observation space
#         self.observation_space = spaces.Box(low=0, high=100, shape=(1,), dtype=np.float32)  # Energy level ranging from 0 to 100

#         # Define the initial energy level
#         self.energy_level = random.randint(50, 80)

#         # Define the resource manager properties
#         self.energy_capacity = 50  # Maximum energy capacity (Decreased from 100 to 50)
#         self.energy_threshold = 20  # Energy threshold for energy-saving mode
#         self.energy_saving_factor = 0.5  # Factor by which energy is saved when in energy-saving mode
#         self.energy_usage = 10  # Increased energy used per time step for temperature changes (from 5 to 10)

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.electricity_cost = 0.1  # Cost per unit of electricity

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a sinusoidal pattern
#         max_solar_radiation = 1000  # Maximum solar radiation in watts per square meter
#         amplitude = max_solar_radiation / 2
#         mean_solar_radiation = max_solar_radiation / 2
#         solar_radiation = amplitude * np.sin(np.pi * (time_of_day / 24)) + mean_solar_radiation
#         return solar_radiation

#     def step(self, action):
#         # Apply the action to change the energy level
#         if action == 0:  # Decrease energy
#             self.energy_level -= self.energy_usage
#         elif action == 1:  # Keep energy level (no change)
#             pass
#         elif action == 2:  # Increase energy
#             # Calculate time-dependent solar radiation
#             solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)

#             # Calculate energy produced by solar panels
#             solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation
#             self.energy_level += solar_energy

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(self.energy_capacity, self.energy_level))

#         # Calculate cost of using outside electricity
#         electricity_cost = 0
#         if action == 2:  # If increasing energy (from outside source)
#             electricity_cost = self.energy_usage * self.electricity_cost

#         # Calculate reward based on energy-saving state and electricity cost
#         if self.energy_level >= self.energy_threshold:
#             reward = 1 - electricity_cost  # Deduct electricity cost from reward
#         else:
#             reward = -1 - electricity_cost  # Deduct electricity cost from reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         return np.array([self.energy_level], dtype=np.float32), reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.energy_level = random.randint(50, 80)
#         self.current_step = 0

#         return np.array([self.energy_level], dtype=np.float32)

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


Removed the direct access to the energy levels, agent changes temp now

In [181]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(3)  # Increase(1), Decrease(-1) or Keep(0) actions

#         # Define the observation space
#         self.observation_space = spaces.Box(low=16, high=33, shape=(1,), dtype=np.float32)  # Temperature range from 16 to 33 degrees Celsius

#         # Define the initial temperature (random between 24 and 28 degrees Celsius)
#         self.initial_temperature = np.random.uniform(24, 28)

#         # Set the current temperature to the initial temperature
#         self.temperature = self.initial_temperature

#         # Define the resource manager properties
#         self.energy_capacity = 50  # Maximum energy capacity (Decreased from 100 to 50)
#         self.energy_threshold = 20  # Energy threshold for energy-saving mode
#         self.energy_saving_factor = 0.5  # Factor by which energy is saved when in energy-saving mode
#         self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.electricity_cost = 1  # Cost per unit of electricity

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a sinusoidal pattern
#         max_solar_radiation = 800  # Maximum solar radiation in watts per square meter
#         amplitude = max_solar_radiation / 2
#         mean_solar_radiation = max_solar_radiation / 2
#         solar_radiation = amplitude * np.sin(np.pi * (time_of_day / 24)) + mean_solar_radiation
#         return solar_radiation

#     def step(self, action):
#         # Apply the action to change the temperature
#         if action == 0:  # Decrease temperature
#             self.temperature -= 1 
#         elif action == 1:  # Keep temperature (no change)
#             pass
#         elif action == 2:  # Increase temperature
#             self.temperature += 1 

#         # Clip temperature within the valid range
#         self.temperature = max(16, min(33, self.temperature))
#         solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)

#         # Calculate energy produced by solar panels
#         solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#         # Calculate energy usage for temperature adjustment (proportional to the magnitude of temperature change)
#         energy_usage = abs(self.temperature - self.initial_temperature)* self.energy_usage_factor

#         net_energy = energy_usage - solar_energy

#         # Clip energy usage within the valid range
#         # energy_usage = min(self.energy_capacity, energy_usage)

#         energy_left = self.energy_capacity - net_energy

#         # Calculate reward based on energy-saving state and electricity cost
#         if  energy_left>= self.energy_threshold:
#             reward = 1 - energy_usage * self.electricity_cost  # Deduct electricity cost from reward
#         else:
#             reward = -1 - energy_usage * self.electricity_cost  # Deduct electricity cost from reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         return np.array([self.temperature], dtype=np.float32), reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.temperature = self.initial_temperature
#         self.current_step = 0

#         return np.array([self.temperature], dtype=np.float32)

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


In [182]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(7)  # Actions: Decrease(1), Keep(2), Increase(3) temperature by 1 degree, Decrease(4), Increase(5) temperature by 2 degrees, Decrease(6), Increase(7) temperature by 3 degrees

#         # Define the observation space
#         self.observation_space = spaces.Box(low=16, high=33, shape=(1,), dtype=np.float32)  # Temperature range from 16 to 33 degrees Celsius

#         # Define the initial temperature (random between 24 and 28 degrees Celsius)
#         self.initial_temperature = np.random.uniform(24, 16)

#         # Set the current temperature to the initial temperature
#         self.temperature = self.initial_temperature

#         # Define the resource manager properties

#         self.energy_capacity = 50  # Maximum energy capacity
#         self.energy_level = 40
#         self.energy_threshold = 20  # Energy threshold for energy-saving mode
#         self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.base_electricity_cost = 1  # Base cost per unit of electricity
#         self.dynamic_electricity_cost = 1  # Dynamic cost multiplier

#         # Initialize outside energy used
#         self.outside_energy_used = 0

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a sinusoidal pattern
#         max_solar_radiation = 800  # Maximum solar radiation in watts per square meter
#         amplitude = max_solar_radiation / 2
#         mean_solar_radiation = max_solar_radiation / 2
#         solar_radiation = amplitude * np.sin(np.pi * (time_of_day / 24)) + mean_solar_radiation
#         return solar_radiation

#     def step(self, action):
#         # Determine the temperature change based on the action
#         temp_change = 0
#         if action == 1:  # Decrease temperature by 1 degree
#             temp_change = -1
#         elif action == 2:  # Keep temperature (no change)
#             temp_change = 0
#         elif action == 3:  # Increase temperature by 1 degree
#             temp_change = 1
#         elif action == 4:  # Decrease temperature by 2 degrees
#             temp_change = -2
#         elif action == 5:  # Increase temperature by 2 degrees
#             temp_change = 2
#         elif action == 6:  # Decrease temperature by 3 degrees
#             temp_change = -3
#         elif action == 7:  # Increase temperature by 3 degrees
#             temp_change = 3

#         # Apply the temperature change
#         self.temperature += temp_change

#         # Clip temperature within the valid range
#         self.temperature = max(16, min(33, self.temperature))

#         # Calculate energy produced by solar panels
#         solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
#         solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#         # Calculate energy usage for temperature adjustment
#         energy_usage =  0.5 *(abs(temp_change)**2) * self.energy_usage_factor

#         # Calculate net energy (energy usage - solar energy)
#         net_energy = energy_usage - solar_energy

#         # Track outside energy used
#         if net_energy < 0:  # If net energy is negative (using too much energy from outside)
#             self.outside_energy_used += abs(net_energy) * self.base_electricity_cost * self.dynamic_electricity_cost  # Add the cost of outside energy used

#         # Clip net energy within the valid range
#         energy_left = self.energy_capacity - max(0, net_energy)

#         # Calculate reward based on energy-saving state and electricity cost
#         if energy_left >= self.energy_threshold:
#             reward = 1 - energy_usage * self.base_electricity_cost * self.dynamic_electricity_cost  # Deduct electricity cost from reward
#         else:
#             reward = -1 - energy_usage * self.base_electricity_cost * self.dynamic_electricity_cost  # Deduct electricity cost from reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         return np.array([self.temperature], dtype=np.float32), reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.temperature = self.initial_temperature
#         self.current_step = 0
#         self.outside_energy_used = 0  # Reset outside energy used

#         return np.array([self.temperature], dtype=np.float32)

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


# Success 1

In [183]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(7)  # Actions: Decrease(1), Keep(2), Increase(3) temperature by 1 degree, Decrease(4), Increase(5) temperature by 2 degrees, Decrease(6), Increase(7) temperature by 3 degrees

#         # Define the observation space
#         self.observation_space = spaces.Box(low=np.array([16, 0]), high=np.array([33, 100]), dtype=np.float32)  # Temperature range from 16 to 33 degrees Celsius, energy level from 0 to 100

#         # Define the initial temperature (random between 24 and 28 degrees Celsius)
#         self.initial_temperature = np.random.uniform(24, 26)

#         # Set the current temperature to the initial temperature
#         self.temperature = self.initial_temperature

#         # Initialize energy level
#         self.energy_level = 60

#         # Define the ideal temperature
#         self.ideal_temperature = 22

#         # Define the resource manager properties
#         self.energy_capacity = 100
#         self.energy_threshold = 30  # Energy threshold for energy-saving mode
#         self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.base_electricity_cost = 1  # Base cost per unit of electricity

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a sinusoidal pattern
#         max_solar_radiation = 800  # Maximum solar radiation in watts per square meter
#         amplitude = max_solar_radiation / 2
#         mean_solar_radiation = max_solar_radiation / 2
#         solar_radiation = amplitude * np.sin(np.pi * (time_of_day / 24)) + mean_solar_radiation
#         return solar_radiation

#     def step(self, action):
#         # Determine the temperature change based on the action
#         temp_change = 0
#         if action == 1:  # Decrease temperature by 1 degree
#             temp_change = -1
#         elif action == 2:  # Keep temperature (no change)
#             temp_change = 0
#         elif action == 3:  # Increase temperature by 1 degree
#             temp_change = 1
#         elif action == 4:  # Decrease temperature by 2 degrees
#             temp_change = -2
#         elif action == 5:  # Increase temperature by 2 degrees
#             temp_change = 2
#         elif action == 6:  # Decrease temperature by 3 degrees
#             temp_change = -3
#         elif action == 7:  # Increase temperature by 3 degrees
#             temp_change = 3

#         # Apply the temperature change
#         self.temperature += temp_change

#         # Clip temperature within the valid range
#         self.temperature = max(16, min(33, self.temperature))

#         # Calculate energy produced by solar panels
#         solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
#         solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#         # Calculate energy usage for temperature adjustment
#         energy_usage = (abs(temp_change)) * self.energy_usage_factor

#         # Calculate net energy (energy usage - solar energy)

#         self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)

#         self.energy_level = self.energy_level - energy_usage 

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(100, self.energy_level))

#         # Calculate reward based on energy-saving state and temperature
#         energy_reward = 0
#         temperature_reward = -abs(self.temperature - self.ideal_temperature)

#         if self.energy_level >= self.energy_threshold:
#             energy_reward = 1  # Reward for maintaining energy above threshold
#         else:
#             energy_reward = -2

#         reward = temperature_reward + energy_reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         # Return the next state, reward, whether the episode is done, and additional info
#         return np.array([self.temperature, self.energy_level], dtype=np.float32), reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.temperature = self.initial_temperature
#         self.energy_level = 50
#         self.current_step = 0

#         # Return the initial state
#         return np.array([self.temperature, self.energy_level], dtype=np.float32)

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


In [184]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(7)  # Actions: Decrease(1), Keep(2), Increase(3) temperature by 1 degree, Decrease(4), Increase(5) temperature by 2 degrees, Decrease(6), Increase(7) temperature by 3 degrees

#         # Define the observation space
#         self.observation_space = spaces.Box(low=np.array([16, 0]), high=np.array([33, 100]), dtype=np.float32)  # Temperature range from 16 to 33 degrees Celsius, energy level from 0 to 100

#         # Define the initial temperature (random between 24 and 26 degrees Celsius)
#         self.initial_temperature = np.random.uniform(24, 26)

#         # Set the current temperature to the initial temperature
#         self.temperature = self.initial_temperature

#         # Initialize energy level
#         self.energy_level = 60

#         # Define the ideal temperature
#         self.ideal_temperature = 22

#         # Define the resource manager properties
#         self.energy_capacity = 100
#         self.energy_threshold = 30  # Energy threshold for energy-saving mode
#         self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.base_electricity_cost = 1  # Base cost per unit of electricity

#         # Initialize solar radiation parameters
#         self.solar_radiation_variation = 100  # Maximum variation in solar radiation
#         self.solar_radiation_mean = 700  # Mean solar radiation during the day
#         self.randomness_factor = 0.1  # Factor to control randomness

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a pattern with variations based on time of day

#         # Define solar radiation ranges for different times of the day
#         if time_of_day < 6:  # Night
#             solar_radiation = np.random.uniform(0, 100)
#         elif time_of_day < 9:  # Early morning
#             solar_radiation = np.random.uniform(500, 700)
#         elif time_of_day < 15:  # Afternoon
#             solar_radiation = np.random.uniform(700, 1000)
#         elif time_of_day < 19:  # Evening
#             solar_radiation = np.random.uniform(500, 700)
#         else:  # Night
#             solar_radiation = 0

#         # Introduce randomness with occasional complete randomness
#         if random.random() < self.randomness_factor:
#             solar_radiation = np.random.uniform(self.solar_radiation_mean - self.solar_radiation_variation,
#                                                  self.solar_radiation_mean + self.solar_radiation_variation)

#         return solar_radiation

#     def step(self, action):
#         # Determine the temperature change based on the action
#         temp_change = 0
#         if action == 1:  # Decrease temperature by 1 degree
#             temp_change = -1
#         elif action == 2:  # Keep temperature (no change)
#             temp_change = 0
#         elif action == 3:  # Increase temperature by 1 degree
#             temp_change = 1
#         elif action == 4:  # Decrease temperature by 2 degrees
#             temp_change = -2
#         elif action == 5:  # Increase temperature by 2 degrees
#             temp_change = 2
#         elif action == 6:  # Decrease temperature by 3 degrees
#             temp_change = -3
#         elif action == 7:  # Increase temperature by 3 degrees
#             temp_change = 3

#         # Apply the temperature change
#         self.temperature += temp_change

#         # Clip temperature within the valid range
#         self.temperature = max(16, min(33, self.temperature))

#         # Calculate energy produced by solar panels
#         solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
#         solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#         # Calculate energy usage for temperature adjustment
#         energy_usage = (abs(temp_change)) * self.energy_usage_factor

#         # Calculate net energy (energy usage - solar energy)
#         self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)
#         self.energy_level = self.energy_level - energy_usage 

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(100, self.energy_level))

#         # Calculate reward based on energy-saving state and temperature
#         energy_reward = 0
#         temperature_reward = -abs(self.temperature - self.ideal_temperature)

#         if self.energy_level >= self.energy_threshold:
#             energy_reward = 1  # Reward for maintaining energy above threshold
#         else:
#             energy_reward = -2

#         reward = temperature_reward + energy_reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         # Return the next state, reward, whether the episode is done, and additional info
#         return np.array([self.temperature, self.energy_level], dtype=np.float32), reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.temperature = self.initial_temperature
#         self.energy_level = 50
#         self.current_step = 0

#         # Return the initial state
#         return np.array([self.temperature, self.energy_level], dtype=np.float32)

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


In [185]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(7)  # Actions: Decrease(1), Keep(2), Increase(3) temperature by 1 degree, Decrease(4), Increase(5) temperature by 2 degrees, Decrease(6), Increase(7) temperature by 3 degrees

#         # Define the observation space
#         self.observation_space = spaces.Box(low=np.array([16, 0]), high=np.array([33, 100]), dtype=np.float32)  # Temperature range from 16 to 33 degrees Celsius, energy level from 0 to 100

#         # Define the initial temperature (random between 24 and 26 degrees Celsius)
#         self.initial_temperature = np.random.uniform(24, 26)

#         # Set the current temperature to the initial temperature
#         self.temperature = self.initial_temperature

#         # Initialize energy level
#         self.energy_level = 60

#         # Define the ideal temperature
#         self.ideal_temperature = 22

#         # Define the resource manager properties
#         self.energy_capacity = 100
#         self.energy_threshold = 30  # Energy threshold for energy-saving mode
#         self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.base_electricity_cost = 1  # Base cost per unit of electricity

#         # Initialize solar radiation parameters
#         self.solar_radiation_variation = 100  # Maximum variation in solar radiation
#         self.solar_radiation_mean = 700  # Mean solar radiation during the day
#         self.randomness_factor = 0.1  # Factor to control randomness

#         # Exploration parameters
#         self.epsilon = 0.1  # Exploration rate
#         self.min_epsilon = 0.01  # Minimum exploration rate
#         self.epsilon_decay = 0.99  # Exploration decay rate

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a pattern with variations based on time of day

#         # Define solar radiation ranges for different times of the day
#         if time_of_day < 6:  # Night
#             solar_radiation = np.random.uniform(0, 100)
#         elif time_of_day < 9:  # Early morning
#             solar_radiation = np.random.uniform(500, 700)
#         elif time_of_day < 15:  # Afternoon
#             solar_radiation = np.random.uniform(700, 1000)
#         elif time_of_day < 19:  # Evening
#             solar_radiation = np.random.uniform(500, 700)
#         else:  # Night
#             solar_radiation = 0

#         # Introduce randomness with occasional complete randomness
#         if random.random() < self.randomness_factor:
#             solar_radiation = np.random.uniform(self.solar_radiation_mean - self.solar_radiation_variation,
#                                                  self.solar_radiation_mean + self.solar_radiation_variation)

#         return solar_radiation

#     def step(self, action):
#         # Determine the temperature change based on the action
#         temp_change = 0
#         if action == 1:  # Decrease temperature by 1 degree
#             temp_change = -1
#         elif action == 2:  # Keep temperature (no change)
#             temp_change = 0
#         elif action == 3:  # Increase temperature by 1 degree
#             temp_change = 1
#         elif action == 4:  # Decrease temperature by 2 degrees
#             temp_change = -2
#         elif action == 5:  # Increase temperature by 2 degrees
#             temp_change = 2
#         elif action == 6:  # Decrease temperature by 3 degrees
#             temp_change = -3
#         elif action == 7:  # Increase temperature by 3 degrees
#             temp_change = 3

#         # Apply the temperature change
#         self.temperature += temp_change

#         # Clip temperature within the valid range
#         self.temperature = max(16, min(33, self.temperature))

#         # Calculate energy produced by solar panels
#         solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
#         solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#         # Calculate energy usage for temperature adjustment
#         energy_usage = (abs(temp_change)) * self.energy_usage_factor

#         # Calculate net energy (energy usage - solar energy)
#         self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)
#         self.energy_level = self.energy_level - energy_usage 

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(100, self.energy_level))

#         # Calculate reward based on energy-saving state and temperature
#         temperature_reward = -abs(self.temperature - self.ideal_temperature)

#         if self.energy_level >= self.energy_threshold:
#             energy_reward = 1  # Reward for maintaining energy above threshold
#         else:
#             energy_reward = -2

#         reward = temperature_reward + energy_reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Exploration decay
#         self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

#         # Choose action with ε-greedy exploration
#         if random.random() < self.epsilon:
#             action = self.action_space.sample()

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         # Return the next state, reward, whether the episode is done, and additional info
#         return np.array([self.temperature, self.energy_level], dtype=np.float32), reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.temperature = self.initial_temperature
#         self.energy_level = 50
#         self.current_step = 0

#         # Return the initial state
#         return np.array([self.temperature, self.energy_level], dtype=np.float32)

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


# Success 2

In [186]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(7)  # Actions: Decrease(1), Keep(2), Increase(3) temperature by 1 degree, Decrease(4), Increase(5) temperature by 2 degrees, Decrease(6), Increase(7) temperature by 3 degrees

#         # Define the observation space with normalized ranges
#         self.observation_space = spaces.Box(low=np.array([-1, -1]), high=np.array([1, 1]), dtype=np.float32)  # Normalized observation space

#         # Define the initial temperature (random between 24 and 26 degrees Celsius)
#         self.initial_temperature = np.random.uniform(24, 26)

#         # Set the current temperature to the initial temperature
#         self.temperature = self.initial_temperature

#         # Initialize energy level
#         self.energy_level = 60

#         # Define the ideal temperature
#         self.ideal_temperature = 22

#         # Define the resource manager properties
#         self.energy_capacity = 100
#         self.energy_threshold = 30  # Energy threshold for energy-saving mode
#         self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.base_electricity_cost = 1  # Base cost per unit of electricity

#         # Initialize solar radiation parameters
#         self.solar_radiation_variation = 100  # Maximum variation in solar radiation
#         self.solar_radiation_mean = 700  # Mean solar radiation during the day
#         self.randomness_factor = 0.1  # Factor to control randomness

#         # Exploration parameters
#         self.epsilon = 0.1  # Exploration rate
#         self.min_epsilon = 0.01  # Minimum exploration rate
#         self.epsilon_decay = 0.99  # Exploration decay rate

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a pattern with variations based on time of day

#         # Define solar radiation ranges for different times of the day
#         if time_of_day < 6:  # Night
#             solar_radiation = np.random.uniform(0, 100)
#         elif time_of_day < 9:  # Early morning
#             solar_radiation = np.random.uniform(500, 700)
#         elif time_of_day < 15:  # Afternoon
#             solar_radiation = np.random.uniform(700, 1000)
#         elif time_of_day < 19:  # Evening
#             solar_radiation = np.random.uniform(500, 700)
#         else:  # Night
#             solar_radiation = 0

#         # Introduce randomness with occasional complete randomness
#         if random.random() < self.randomness_factor:
#             solar_radiation = np.random.uniform(self.solar_radiation_mean - self.solar_radiation_variation,
#                                                  self.solar_radiation_mean + self.solar_radiation_variation)

#         return solar_radiation

#     def step(self, action):
#         # Determine the temperature change based on the action
#         temp_change = 0
#         if action == 1:  # Decrease temperature by 1 degree
#             temp_change = -1
#         elif action == 2:  # Keep temperature (no change)
#             temp_change = 0
#         elif action == 3:  # Increase temperature by 1 degree
#             temp_change = 1
#         elif action == 4:  # Decrease temperature by 2 degrees
#             temp_change = -2
#         elif action == 5:  # Increase temperature by 2 degrees
#             temp_change = 2
#         elif action == 6:  # Decrease temperature by 3 degrees
#             temp_change = -3
#         elif action == 7:  # Increase temperature by 3 degrees
#             temp_change = 3

#         # Apply the temperature change
#         self.temperature += temp_change

#         # Clip temperature within the valid range
#         self.temperature = max(16, min(33, self.temperature))

#         # Calculate energy produced by solar panels
#         solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
#         solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#         # Calculate energy usage for temperature adjustment
#         energy_usage = (abs(temp_change)) * self.energy_usage_factor

#         # Calculate net energy (energy usage - solar energy)
#         self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)
#         self.energy_level = self.energy_level - energy_usage 

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(100, self.energy_level))

#         # Calculate reward based on energy-saving state and temperature
#         temperature_reward = -0.5*(abs(self.temperature - self.ideal_temperature)**2)

#         if self.energy_level >= self.energy_threshold:
#             energy_reward = 1  # Reward for maintaining energy above threshold
#         else:
#             energy_reward = -1

#         reward = temperature_reward + energy_reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Exploration decay
#         self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

#         # Choose action with ε-greedy exploration
#         if random.random() < self.epsilon:
#             action = self.action_space.sample()

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         # Normalize observation space
#         normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / 100])

#         # Return the next state, reward, whether the episode is done, and additional info
#         return normalized_observation, reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.temperature = self.initial_temperature
#         self.energy_level = 60
#         self.current_step = 0

#         # Normalize observation space
#         normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / 100])

#         # Return the initial state
#         return normalized_observation

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


In [187]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(28)  # Total number of actions: 7 temperature adjustments * 4 airflow adjustments
        
#         # Define the observation space with normalized ranges
#         self.observation_space = spaces.Box(low=np.array([-1, -1]), high=np.array([1, 1]), dtype=np.float32)  # Normalized observation space: temperature and energy level

#         # Define the initial temperature (random between 24 and 26 degrees Celsius)
#         self.initial_temperature = np.random.uniform(24, 26)

#         # Set the current temperature to the initial temperature
#         self.temperature = self.initial_temperature

#         # Initialize energy level
#         self.energy_level = 60

#         # Define the ideal temperature
#         self.ideal_temperature = 22

#         # Define the resource manager properties
#         self.energy_capacity = 100
#         self.energy_threshold = 30  # Energy threshold for energy-saving mode
#         self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment
#         self.base_airflow = 0  # Base airflow without energy consumption
#         self.max_airflow = 3  # Maximum airflow that can be achieved with energy consumption

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.base_electricity_cost = 1  # Base cost per unit of electricity

#         # Initialize solar radiation parameters
#         self.solar_radiation_variation = 100  # Maximum variation in solar radiation
#         self.solar_radiation_mean = 700  # Mean solar radiation during the day
#         self.randomness_factor = 0.1  # Factor to control randomness

#         # Exploration parameters
#         self.epsilon = 0.1  # Exploration rate
#         self.min_epsilon = 0.01  # Minimum exploration rate
#         self.epsilon_decay = 0.99  # Exploration decay rate

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a pattern with variations based on time of day

#         # Define solar radiation ranges for different times of the day
#         if time_of_day < 6:  # Night
#             solar_radiation = np.random.uniform(0, 100)
#         elif time_of_day < 9:  # Early morning
#             solar_radiation = np.random.uniform(500, 700)
#         elif time_of_day < 15:  # Afternoon
#             solar_radiation = np.random.uniform(700, 1000)
#         elif time_of_day < 19:  # Evening
#             solar_radiation = np.random.uniform(500, 700)
#         else:  # Night
#             solar_radiation = 0

#         # Introduce randomness with occasional complete randomness
#         if random.random() < self.randomness_factor:
#             solar_radiation = np.random.uniform(self.solar_radiation_mean - self.solar_radiation_variation,
#                                                  self.solar_radiation_mean + self.solar_radiation_variation)

#         return solar_radiation

#     def step(self, action):
#         temperature_action = action // 4 + 1  # Temperature adjustment
#         airflow_action = action % 4  # Airflow adjustment

#         # Determine the temperature change based on the action
#         temp_change = 0
#         if temperature_action == 1:  # Decrease temperature by 1 degree
#             temp_change = -1
#         elif temperature_action == 2:  # Keep temperature (no change)
#             temp_change = 0
#         elif temperature_action == 3:  # Increase temperature by 1 degree
#             temp_change = 1
#         elif temperature_action == 4:  # Decrease temperature by 2 degrees
#             temp_change = -2
#         elif temperature_action == 5:  # Increase temperature by 2 degrees
#             temp_change = 2
#         elif temperature_action == 6:  # Decrease temperature by 3 degrees
#             temp_change = -3
#         elif temperature_action == 7:  # Increase temperature by 3 degrees
#             temp_change = 3

#         # Apply the temperature change
#         self.temperature += temp_change

#         # Clip temperature within the valid range
#         self.temperature = max(16, min(33, self.temperature))

#         # Calculate energy produced by solar panels
#         solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
#         solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#         # Calculate energy usage for temperature adjustment
#         energy_usage = (abs(temp_change)) * self.energy_usage_factor

#         # Adjust airflow based on the action
#         airflow = self.base_airflow + airflow_action * (self.max_airflow / (self.action_space.n // 4 - 1))

#         # Calculate net energy (energy usage - solar energy)
#         self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)
#         self.energy_level -= energy_usage

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(100, self.energy_level))

#         # Calculate reward based on energy-saving state and temperature
#         temperature_reward = -0.5 * (abs(self.temperature - self.ideal_temperature) ** 2)

#         if self.energy_level >= self.energy_threshold:
#             energy_reward = 1  # Reward for maintaining energy above threshold
#         else:
#             energy_reward = -1

#         reward = temperature_reward + energy_reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Exploration decay
#         self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

#         # Choose action with ε-greedy exploration
#         if random.random() < self.epsilon:
#             action = self.action_space.sample()

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         # Normalize observation space
#         normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / 100])

#         # Return the next state, reward, whether the episode is done, and additional info
#         return normalized_observation, reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.temperature = self.initial_temperature
#         self.energy_level = 60
#         self.current_step = 0

#         # Normalize observation space
#         normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / 100])

#         # Return the initial state
#         return normalized_observation

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


In [188]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(28)  # Total number of actions: 7 temperature adjustments * 4 airflow adjustments
        
#         # Define the observation space with normalized ranges
#         self.observation_space = spaces.Box(low=np.array([-1, -1]), high=np.array([1, 1]), dtype=np.float32)  # Normalized observation space: temperature and energy level

#         # Define the initial temperature (random between 24 and 26 degrees Celsius)
#         self.initial_temperature = np.random.uniform(24, 26)

#         # Set the current temperature to the initial temperature
#         self.temperature = self.initial_temperature

#         # Initialize energy level
#         self.energy_level = 60

#         # Define the ideal temperature
#         self.ideal_temperature = 22

#         # Define the resource manager properties
#         self.energy_capacity = 100
#         self.energy_threshold = 30  # Energy threshold for energy-saving mode
#         self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment
#         self.base_airflow = 0  # Base airflow without energy consumption
#         self.max_airflow = 3  # Maximum airflow that can be achieved with energy consumption
#         self.airflow_energy_consumption = 5  # Energy consumption per unit increase in airflow

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.base_electricity_cost = 1  # Base cost per unit of electricity

#         # Initialize solar radiation parameters
#         self.solar_radiation_variation = 100  # Maximum variation in solar radiation
#         self.solar_radiation_mean = 700  # Mean solar radiation during the day
#         self.randomness_factor = 0.1  # Factor to control randomness

#         # Exploration parameters
#         self.epsilon = 0.1  # Exploration rate
#         self.min_epsilon = 0.01  # Minimum exploration rate
#         self.epsilon_decay = 0.99  # Exploration decay rate

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a pattern with variations based on time of day

#         # Define solar radiation ranges for different times of the day
#         if time_of_day < 6:  # Night
#             solar_radiation = np.random.uniform(0, 100)
#         elif time_of_day < 9:  # Early morning
#             solar_radiation = np.random.uniform(500, 700)
#         elif time_of_day < 15:  # Afternoon
#             solar_radiation = np.random.uniform(700, 1000)
#         elif time_of_day < 19:  # Evening
#             solar_radiation = np.random.uniform(500, 700)
#         else:  # Night
#             solar_radiation = 0

#         # Introduce randomness with occasional complete randomness
#         if random.random() < self.randomness_factor:
#             solar_radiation = np.random.uniform(self.solar_radiation_mean - self.solar_radiation_variation,
#                                                  self.solar_radiation_mean + self.solar_radiation_variation)

#         return solar_radiation

#     def step(self, action):
#         temperature_action = action // 4 + 1  # Temperature adjustment
#         airflow_action = action % 4  # Airflow adjustment

#         # Determine the temperature change based on the action
#         temp_change = 0
#         if temperature_action == 1:  # Decrease temperature by 1 degree
#             temp_change = -1
#         elif temperature_action == 2:  # Keep temperature (no change)
#             temp_change = 0
#         elif temperature_action == 3:  # Increase temperature by 1 degree
#             temp_change = 1
#         elif temperature_action == 4:  # Decrease temperature by 2 degrees
#             temp_change = -2
#         elif temperature_action == 5:  # Increase temperature by 2 degrees
#             temp_change = 2
#         elif temperature_action == 6:  # Decrease temperature by 3 degrees
#             temp_change = -3
#         elif temperature_action == 7:  # Increase temperature by 3 degrees
#             temp_change = 3

#         # Apply the temperature change
#         self.temperature += temp_change

#         # Clip temperature within the valid range
#         self.temperature = max(16, min(33, self.temperature))

#         # Calculate energy produced by solar panels
#         solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
#         solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#         # Calculate energy usage for temperature adjustment
#         energy_usage = (abs(temp_change)) * self.energy_usage_factor

#         # Adjust airflow based on the action
#         airflow = self.base_airflow + airflow_action * (self.max_airflow / (self.action_space.n // 4 - 1))

#         # Calculate energy consumption for airflow adjustment
#         airflow_energy_usage = airflow_action * self.airflow_energy_consumption

#         # Calculate net energy (energy usage - solar energy)
#         self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)
#         self.energy_level -= (energy_usage + airflow_energy_usage)

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(100, self.energy_level))

#         # Calculate reward based on energy-saving state and temperature
#         temperature_reward = -0.5 * (abs(self.temperature - self.ideal_temperature) ** 2)

#         if self.energy_level >= self.energy_threshold:
#             energy_reward = 1  # Reward for maintaining energy above threshold
#         else:
#             energy_reward = -1

#         reward = temperature_reward + energy_reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Exploration decay
#         self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

#         # Choose action with ε-greedy exploration
#         if random.random() < self.epsilon:
#             action = self.action_space.sample()

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         # Normalize observation space
#         normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / 100])

#         # Return the next state, reward, whether the episode is done, and additional info
#         return normalized_observation, reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.temperature = self.initial_temperature
#         self.energy_level = 60
#         self.current_step = 0

#         # Normalize observation space
#         normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / 100])

#         # Return the initial state
#         return normalized_observation

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


In [189]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(28)  # Total number of actions: 7 temperature adjustments * 4 airflow adjustments
        
#         # Define the observation space with normalized ranges
#         self.observation_space = spaces.Box(low=np.array([-1, -1]), high=np.array([1, 1]), dtype=np.float32)  # Normalized observation space: temperature and energy level

#         # Define the initial temperature (random between 24 and 26 degrees Celsius)
#         self.initial_temperature = np.random.uniform(24, 26)

#         # Set the current temperature to the initial temperature
#         self.temperature = self.initial_temperature

#         # Initialize energy level
#         self.energy_level = 60

#         # Define the ideal temperature
#         self.ideal_temperature = 22

#         # Define the resource manager properties
#         self.energy_capacity = 100
#         self.energy_threshold = 30  # Energy threshold for energy-saving mode
#         self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment
#         self.base_airflow = 0  # Base airflow without energy consumption
#         self.max_airflow = 3  # Maximum airflow that can be achieved with energy consumption
#         self.airflow_energy_consumption = 5  # Energy consumption per unit increase in airflow

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.base_electricity_cost = 1  # Base cost per unit of electricity

#         # Initialize solar radiation parameters
#         self.solar_radiation_variation = 100  # Maximum variation in solar radiation
#         self.solar_radiation_mean = 700  # Mean solar radiation during the day
#         self.randomness_factor = 0.1  # Factor to control randomness

#         # Define the maximum number of people in the room
#         self.max_people = 5

#         # Initialize the number of people in the room
#         self.num_people = 0

#         # Exploration parameters
#         self.epsilon = 0.1  # Exploration rate
#         self.min_epsilon = 0.01  # Minimum exploration rate
#         self.epsilon_decay = 0.99  # Exploration decay rate

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a pattern with variations based on time of day

#         # Define solar radiation ranges for different times of the day
#         if time_of_day < 6:  # Night
#             solar_radiation = np.random.uniform(0, 100)
#         elif time_of_day < 9:  # Early morning
#             solar_radiation = np.random.uniform(500, 700)
#         elif time_of_day < 15:  # Afternoon
#             solar_radiation = np.random.uniform(700, 1000)
#         elif time_of_day < 19:  # Evening
#             solar_radiation = np.random.uniform(500, 700)
#         else:  # Night
#             solar_radiation = 0

#         # Introduce randomness with occasional complete randomness
#         if random.random() < self.randomness_factor:
#             solar_radiation = np.random.uniform(self.solar_radiation_mean - self.solar_radiation_variation,
#                                                  self.solar_radiation_mean + self.solar_radiation_variation)

#         return solar_radiation

#     def step(self, action):
#         temperature_action = action // 4 + 1  # Temperature adjustment
#         airflow_action = action % 4  # Airflow adjustment

#         # Determine the temperature change based on the action
#         temp_change = 0
#         if temperature_action == 1:  # Decrease temperature by 1 degree
#             temp_change = -1
#         elif temperature_action == 2:  # Keep temperature (no change)
#             temp_change = 0
#         elif temperature_action == 3:  # Increase temperature by 1 degree
#             temp_change = 1
#         elif temperature_action == 4:  # Decrease temperature by 2 degrees
#             temp_change = -2
#         elif temperature_action == 5:  # Increase temperature by 2 degrees
#             temp_change = 2
#         elif temperature_action == 6:  # Decrease temperature by 3 degrees
#             temp_change = -3
#         elif temperature_action == 7:  # Increase temperature by 3 degrees
#             temp_change = 3

#         # Apply the temperature change
#         self.temperature += temp_change

#         # Clip temperature within the valid range
#         self.temperature = max(16, min(33, self.temperature))

#         # Calculate energy produced by solar panels
#         solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
#         solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#         # Calculate energy usage for temperature adjustment
#         energy_usage = (abs(temp_change)) * self.energy_usage_factor

#         # Adjust airflow based on the action and number of people in the room
#         airflow = self.base_airflow + airflow_action * (self.max_airflow / (self.action_space.n // 4 - 1))
#         airflow_energy_usage = 0  # Initialize airflow energy usage
        
#         # Adjust airflow energy consumption based on the number of people in the room
#         if self.num_people > 0:
#             airflow_energy_usage = self.airflow_energy_consumption * self.num_people

#         # Calculate net energy (energy usage - solar energy)
#         self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)
#         self.energy_level -= (energy_usage + airflow_energy_usage)

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(100, self.energy_level))

#         # Calculate reward based on energy-saving state and temperature
#         temperature_reward = -0.5 * (abs(self.temperature - self.ideal_temperature) ** 2)

#         if self.energy_level >= self.energy_threshold:
#             energy_reward = 1  # Reward for maintaining energy above threshold
#         else:
#             energy_reward = -1

#         reward = temperature_reward + energy_reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Exploration decay
#         self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

#         # Choose action with ε-greedy exploration
#         if random.random() < self.epsilon:
#             action = self.action_space.sample()

#         # Additional info can be an empty dictionary
#         info = {}
#         truncated = False

#         # Normalize observation space
#         normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / 100])

#         # Return the next state, reward, whether the episode is done, and additional info
#         return normalized_observation, reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.temperature = self.initial_temperature
#         # self.energy_level = 60
#         self.current_step = 0

#         # Randomly generate the number of people in the room
#         self.num_people = random.randint(0, self.max_people)

#         # Normalize observation space
#         normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / 100])

#         # Return the initial state
#         return normalized_observation

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


In [190]:
# import numpy as np
# import random
# import gym
# import gym.spaces as spaces

# class EnergySavingEnv(gym.Env):
#     def __init__(self):
#         # Define the action space
#         self.action_space = spaces.Discrete(28)  # Total number of actions: 7 temperature adjustments * 4 airflow adjustments
        
#         # Define the observation space with normalized ranges
#         self.observation_space = spaces.Box(low=np.array([-1, -1]), high=np.array([1, 1]), dtype=np.float32)  # Normalized observation space: temperature and energy level

#         # Define the initial temperature (random between 24 and 26 degrees Celsius)
#         self.initial_temperature = np.random.uniform(24, 26)

#         # Set the current temperature to the initial temperature
#         self.temperature = self.initial_temperature

#         # Initialize energy level
#         self.energy_level = 60

#         # Define the ideal temperature
#         self.ideal_temperature = 22

#         # Define the resource manager properties
#         self.energy_capacity = 100
#         self.energy_threshold = 30  # Energy threshold for energy-saving mode
#         self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment
#         self.base_airflow = 0  # Base airflow without energy consumption
#         self.max_airflow = 3  # Maximum airflow that can be achieved with energy consumption
#         self.airflow_energy_consumption = 5  # Energy consumption per unit increase in airflow

#         # Define the number of time steps for the resource manager
#         self.time_steps = 100

#         # Initialize the current time step
#         self.current_step = 0

#         # Add solar energy parameters
#         self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
#         self.solar_panel_area = 25  # Area covered by solar panels in square meters

#         # Define outside electricity cost
#         self.base_electricity_cost = 1  # Base cost per unit of electricity

#         # Initialize solar radiation parameters
#         self.solar_radiation_variation = 100  # Maximum variation in solar radiation
#         self.solar_radiation_mean = 700  # Mean solar radiation during the day
#         self.randomness_factor = 0.1  # Factor to control randomness

#         # Define the maximum number of people in the room
#         self.max_people = 5

#         # Initialize the number of people in the room
#         self.num_people = 0

#         # Exploration parameters
#         self.epsilon = 0.1  # Exploration rate
#         self.min_epsilon = 0.01  # Minimum exploration rate
#         self.epsilon_decay = 0.99  # Exploration decay rate

#     def get_solar_radiation(self, time_of_day):
#         # Simulate time-dependent solar radiation
#         # For simplicity, we'll assume a pattern with variations based on time of day

#         # Define solar radiation ranges for different times of the day
#         if time_of_day < 6:  # Night
#             solar_radiation = np.random.uniform(0, 100)
#         elif time_of_day < 9:  # Early morning
#             solar_radiation = np.random.uniform(500, 700)
#         elif time_of_day < 15:  # Afternoon
#             solar_radiation = np.random.uniform(700, 1000)
#         elif time_of_day < 19:  # Evening
#             solar_radiation = np.random.uniform(500, 700)
#         else:  # Night
#             solar_radiation = 0

#         # Introduce randomness with occasional complete randomness
#         if random.random() < self.randomness_factor:
#             solar_radiation = np.random.uniform(self.solar_radiation_mean - self.solar_radiation_variation,
#                                                  self.solar_radiation_mean + self.solar_radiation_variation)

#         return solar_radiation

#     def step(self, action):
#         temperature_action = action // 4 + 1  # Temperature adjustment
#         airflow_action = action % 4  # Airflow adjustment

#         # Determine the temperature change based on the action
#         temp_change = 0
#         if temperature_action == 1:  # Decrease temperature by 1 degree
#             temp_change = -1
#         elif temperature_action == 2:  # Keep temperature (no change)
#             temp_change = 0
#         elif temperature_action == 3:  # Increase temperature by 1 degree
#             temp_change = 1
#         elif temperature_action == 4:  # Decrease temperature by 2 degrees
#             temp_change = -2
#         elif temperature_action == 5:  # Increase temperature by 2 degrees
#             temp_change = 2
#         elif temperature_action == 6:  # Decrease temperature by 3 degrees
#             temp_change = -3
#         elif temperature_action == 7:  # Increase temperature by 3 degrees
#             temp_change = 3

#         # Apply the temperature change
#         self.temperature += temp_change

#         # Clip temperature within the valid range
#         self.temperature = max(16, min(33, self.temperature))

#         # Calculate energy produced by solar panels
#         solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
#         solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

#         # Calculate energy usage for temperature adjustment
#         energy_usage = (abs(temp_change)) * self.energy_usage_factor

#         # Adjust airflow based on the action and number of people in the room
#         airflow = self.base_airflow + airflow_action * (self.max_airflow / (self.action_space.n // 4 - 1))
#         airflow_energy_usage = 0  # Initialize airflow energy usage
        
#         # Adjust airflow energy consumption based on the number of people in the room
#         if self.num_people > 0:
#             airflow_energy_usage = self.airflow_energy_consumption * self.num_people

#         # Calculate net energy (energy usage - solar energy)
#         self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)
#         self.energy_level -= (energy_usage + airflow_energy_usage)

#         # Clip energy level within the valid range
#         self.energy_level = max(0, min(100, self.energy_level))

#         # Calculate reward based on energy-saving state and temperature
#         temperature_reward = -0.5 * (abs(self.temperature - self.ideal_temperature) ** 2)

#         if self.energy_level >= self.energy_threshold:
#             energy_reward = 1  # Reward for maintaining energy above threshold
#         else:
#             energy_reward = -1

#         reward = temperature_reward + energy_reward

#         # Check if the episode is done
#         done = self.current_step >= self.time_steps

#         # Increment the current time step
#         self.current_step += 1

#         # Exploration decay
#         self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

#         # Choose action with ε-greedy exploration
#         if random.random() < self.epsilon:
#             action = self.action_space.sample()

#         # Additional info can be an empty dictionary
#         info = {}

#         # Normalize observation space
#         normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / 100])

#         # Return the next state, reward, whether the episode is done, and additional info
#         return normalized_observation, reward, done, info

#     def reset(self):
#         # Reset the environment to the initial state
#         self.current_step = 0

#         # Randomly generate the number of people in the room
#         self.num_people = random.randint(0, self.max_people)

#         # Normalize observation space
#         normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / 100])

#         # Return the initial state
#         return normalized_observation

#     def render(self, mode='human'):
#         # Implement visualization (optional)
#         pass


# Room Env

In [191]:
import numpy as np
import random
import gym
import gym.spaces as spaces

class EnergySavingEnv(gym.Env):
    def __init__(self):
        # Define the action space
        self.action_space = spaces.Discrete(28)  # Total number of actions: 7 temperature adjustments * 4 airflow adjustments
        
        # Define the observation space with normalized ranges
        self.observation_space = spaces.Box(low=np.array([-1, -1]), high=np.array([1, 1]), dtype=np.float32)  # Normalized observation space: temperature and energy level

        # Define the initial temperature (random between 24 and 26 degrees Celsius)
        self.initial_temperature = np.random.uniform(24, 26)

        # Set the current temperature to the initial temperature
        self.temperature = self.initial_temperature

        # Initialize energy level
        self.energy_level = 60

        # Define the ideal temperature
        self.ideal_temperature = 22

        # Define the resource manager properties
        self.energy_capacity = 100
        self.energy_threshold = 30  # Energy threshold for energy-saving mode
        self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment
        self.base_airflow = 0  # Base airflow without energy consumption
        self.max_airflow = 3  # Maximum airflow that can be achieved with energy consumption
        self.airflow_energy_consumption = 5  # Energy consumption per unit increase in airflow

        # Define the number of time steps for the resource manager
        self.time_steps = 100

        # Initialize the current time step
        self.current_step = 0

        self.cleaning_energy_consumption = 5  # Energy consumed during home cleaning

        # Define the frequency of home cleaning (once a day/episode)
        self.cleaning_frequency = 1

        # Add solar energy parameters
        self.solar_panel_efficiency = 0.18  # Efficiency of solar panels
        self.solar_panel_area = 25  # Area covered by solar panels in square meters

        # Define outside electricity cost
        self.base_electricity_cost = 1  # Base cost per unit of electricity

        # Initialize solar radiation parameters
        self.solar_radiation_variation = 100  # Maximum variation in solar radiation
        self.solar_radiation_mean = 700  # Mean solar radiation during the day
        self.randomness_factor = 0.1  # Factor to control randomness

        # Define the maximum number of people in the room
        self.max_people = 5

        # Initialize the number of people in the room
        self.num_people = 0

        # Define the frequency of adding/removing people (every 20 time steps)
        self.add_remove_frequency = 25

        # Exploration parameters
        self.epsilon = 0.1  # Exploration rate
        self.min_epsilon = 0.01  # Minimum exploration rate
        self.epsilon_decay = 0.99  # Exploration decay rate

    def get_solar_radiation(self, time_of_day):
        # Simulate time-dependent solar radiation
        # For simplicity, we'll assume a pattern with variations based on time of day

        # Define solar radiation ranges for different times of the day
        if time_of_day < 6:  # Night
            solar_radiation = np.random.uniform(0, 100)
        elif time_of_day < 9:  # Early morning
            solar_radiation = np.random.uniform(500, 700)
        elif time_of_day < 15:  # Afternoon
            solar_radiation = np.random.uniform(700, 1000)
        elif time_of_day < 19:  # Evening
            solar_radiation = np.random.uniform(500, 700)
        else:  # Night
            solar_radiation = 0

        # Introduce randomness with occasional complete randomness
        if random.random() < self.randomness_factor:
            solar_radiation = np.random.uniform(self.solar_radiation_mean - self.solar_radiation_variation,
                                                 self.solar_radiation_mean + self.solar_radiation_variation)

        return solar_radiation

    def step(self, action):
        temperature_action = action // 4 + 1  # Temperature adjustment
        airflow_action = action % 4  # Airflow adjustment

        # Determine the temperature change based on the action
        temp_change = 0
        if temperature_action == 1:  # Decrease temperature by 1 degree
            temp_change = -1
        elif temperature_action == 2:  # Keep temperature (no change)
            temp_change = 0
        elif temperature_action == 3:  # Increase temperature by 1 degree
            temp_change = 1
        elif temperature_action == 4:  # Decrease temperature by 2 degrees
            temp_change = -2
        elif temperature_action == 5:  # Increase temperature by 2 degrees
            temp_change = 2
        elif temperature_action == 6:  # Decrease temperature by 3 degrees
            temp_change = -3
        elif temperature_action == 7:  # Increase temperature by 3 degrees
            temp_change = 3

        # Apply the temperature change
        self.temperature += temp_change

        # Clip temperature within the valid range
        self.temperature = max(16, min(33, self.temperature))

        # Calculate energy produced by solar panels
        solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
        solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

        # Calculate energy usage for temperature adjustment
        energy_usage = (abs(temp_change)) * self.energy_usage_factor

        # Adjust airflow based on the action and number of people in the room
        airflow = self.base_airflow + airflow_action * (self.max_airflow / (self.action_space.n // 4 - 1))
        airflow_energy_usage = 0  # Initialize airflow energy usage
        
        # Adjust airflow energy consumption based on the number of people in the room
        if self.num_people > 0:
            airflow_energy_usage = self.airflow_energy_consumption * self.num_people

        # Calculate net energy (energy usage - solar energy)
        self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)
        self.energy_level -= (energy_usage + airflow_energy_usage)

        # Clip energy level within the valid range
        self.energy_level = max(0, min(self.energy_capacity, self.energy_level))

        # Calculate reward based on energy-saving state and temperature
        temperature_reward = -0.5 * (abs(self.temperature - self.ideal_temperature) ** 2)

        if self.energy_level >= self.energy_threshold:
            energy_reward = 1  # Reward for maintaining energy above threshold
        else:
            energy_reward = -1

        reward = temperature_reward + energy_reward

        # Check if the episode is done
        done = self.current_step >= self.time_steps

        # Increment the current time step
        self.current_step += 1

        # Add or remove people from the room every 20 time steps
        if self.current_step % self.add_remove_frequency == 0:
            if random.random() < 0.5:  # Randomly decide whether to add or remove people
                if self.num_people < self.max_people:
                    self.num_people += 1
            else:
                if self.num_people > 0:
                    self.num_people -= 1

        # Exploration decay
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

        # Choose action with ε-greedy exploration
        if random.random() < self.epsilon:
            action = self.action_space.sample()

        # Additional info can be an empty dictionary
        info = {}

        # Normalize observation space
        normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / self.energy_capacity])

        # Return the next state, reward, whether the episode is done, and additional info
        return normalized_observation, reward, done, info

    def reset(self):
        # Reset the current time step
        self.current_step = 0
        
        # Randomly generate the number of people in the room
        self.num_people = random.randint(0, self.max_people)

        # Check if it's time for home cleaning
        if self.current_step % self.cleaning_frequency == 0:
            # Consume energy for home cleaning
            self.energy_level -= self.cleaning_energy_consumption

            # Clip energy level within the valid range
            self.energy_level = max(0, min(self.energy_capacity, self.energy_level))

        # Normalize observation space
        normalized_observation = np.array([(self.temperature - 16) / 17, self.energy_level / self.energy_capacity])

        # Return the initial state
        return normalized_observation

# House Env

In [192]:
import numpy as np
import random
import gym
import gym.spaces as spaces

class EnergySavingEnv(gym.Env):
    def __init__(self, num_bedrooms=3, num_people_per_bedroom=[1,2,1], num_people_in_living_room=2):
        self.num_bedrooms = num_bedrooms
        self.num_people_per_bedroom = num_people_per_bedroom if num_people_per_bedroom is not None else [2] * num_bedrooms
        self.num_people_in_living_room = num_people_in_living_room

        # Define the action space
        self.action_space = spaces.Discrete(28)  # Total number of actions: 7 temperature adjustments * 4 airflow adjustments
        
        # Define the observation space with normalized ranges
        num_rooms = num_bedrooms + 1  # Include living room as well
        self.observation_space = spaces.Box(low=-1, high=1, shape=(num_rooms+1,1), dtype=np.float32)

        # Define the initial temperature (random between 24 and 26 degrees Celsius) for each room
        self.initial_temperatures = np.random.uniform(24, 26, size=(num_rooms,))

        # Set the current temperature to the initial temperature
        self.temperatures = self.initial_temperatures.copy()

        # Initialize energy level for the whole house
        self.energy_level = np.array([120])

        # Define the ideal temperature for each room
        self.ideal_temperatures = [22] * num_rooms  # Assuming ideal temperature of 22 for all rooms

        # Define the resource manager properties
        self.energy_capacity = 250
        self.energy_threshold = 50  # Energy threshold for energy-saving mode
        self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment
        self.base_airflow = 0  # Base airflow without energy consumption
        self.max_airflow = 3  # Maximum airflow that can be achieved with energy consumption
        self.airflow_energy_consumption = 0.5  # Energy consumption per unit increase in airflow

        # Define the number of time steps for the resource manager
        self.time_steps = 100

        # Initialize the current time step
        self.current_step = 0

        self.cleaning_energy_consumption = 1  # Energy consumed during home cleaning

        # Define the frequency of home cleaning (once a day/episode)
        self.cleaning_frequency = 1

        # Add solar energy parameters
        self.solar_panel_efficiency = 0.2  # Efficiency of solar panels
        self.solar_panel_area = 100  # Area covered by solar panels in square meters

        # Define outside electricity cost
        self.base_electricity_cost = 1  # Base cost per unit of electricity

        # Initialize solar radiation parameters
        self.solar_radiation_variation = 100  # Maximum variation in solar radiation
        self.solar_radiation_mean = 700  # Mean solar radiation during the day
        self.randomness_factor = 0.1  # Factor to control randomness

        # Define the maximum number of people in each room
        self.max_people_per_room = 3
        self.num_people_in_rooms = self.num_people_per_bedroom.copy() + [0] * (num_bedrooms + 1)  # Initialize with specified people in bedrooms
        self.num_people_in_rooms[-1] = num_people_in_living_room  # Set the number of people in the living room

        # Define the frequency of adding/removing people (every 20 time steps)
        self.add_remove_frequency = 25

        # Exploration parameters
        self.epsilon = 0.1 # Exploration rate
        self.min_epsilon = 0.01  # Minimum exploration rate
        self.epsilon_decay = 0.99  # Exploration decay rate

    def get_solar_radiation(self, time_of_day):
        # Simulate time-dependent solar radiation
        # For simplicity, we'll assume a pattern with variations based on time of day

        # Define solar radiation ranges for different times of the day
        if time_of_day < 6:  # Night
            solar_radiation = np.random.uniform(0, 100)
        elif time_of_day < 9:  # Early morning
            solar_radiation = np.random.uniform(500, 700)
        elif time_of_day < 15:  # Afternoon
            solar_radiation = np.random.uniform(700, 1000)
        elif time_of_day < 19:  # Evening
            solar_radiation = np.random.uniform(500, 700)
        else:  # Night
            solar_radiation = 0

        # Introduce randomness with occasional complete randomness
        if random.random() < self.randomness_factor:
            solar_radiation = np.random.uniform(self.solar_radiation_mean - self.solar_radiation_variation,
                                                 self.solar_radiation_mean + self.solar_radiation_variation)

        return solar_radiation

    def step(self, action):
        temperature_action = action // 4 + 1  # Temperature adjustment
        airflow_action = action % 4  # Airflow adjustment

        # Determine the temperature change based on the action
        temp_changes = np.zeros_like(self.temperatures)
        temp_change = 0
        if temperature_action == 1:  # Decrease temperature by 1 degree
            temp_change = -1
        elif temperature_action == 2:  # Keep temperature (no change)
            temp_change = 0
        elif temperature_action == 3:  # Increase temperature by 1 degree
            temp_change = 1
        elif temperature_action == 4:  # Decrease temperature by 2 degrees
            temp_change = -2
        elif temperature_action == 5:  # Increase temperature by 2 degrees
            temp_change = 2
        elif temperature_action == 6:  # Decrease temperature by 3 degrees
            temp_change = -3
        elif temperature_action == 7:  # Increase temperature by 3 degrees
            temp_change = 3

        # Apply the temperature change to all rooms
        temp_changes[:self.num_bedrooms] = temp_change

        # Clip temperature within the valid range
        self.temperatures += temp_changes
        self.temperatures = np.clip(self.temperatures, 16, 33)

        # Calculate energy produced by solar panels
        solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
        solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

        # Calculate energy usage for temperature adjustment
        energy_usage = np.abs(temp_changes) * self.energy_usage_factor

        # Adjust airflow based on the action and number of people in the room
        airflow = self.base_airflow + airflow_action * (self.max_airflow / (self.action_space.n // 4 - 1))
        airflow_energy_usage = airflow * self.airflow_energy_consumption  # Initialize airflow energy usage
        
        # Adjust airflow energy consumption based on the number of people in each room
        for room_idx, num_people in enumerate(self.num_people_in_rooms):
            if num_people > 0:
                airflow_energy_usage += self.airflow_energy_consumption * num_people

        # Calculate net energy (energy usage - solar energy)
        self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)
        self.energy_level -= (np.sum(energy_usage) + airflow_energy_usage)

        # Clip energy level within the valid range
        self.energy_level = np.clip(self.energy_level, 0, self.energy_capacity)

        # Calculate reward based on energy-saving state and temperature
        # temperature_rewards = -0.5 * (np.abs(self.temperatures - self.ideal_temperatures) ** 2)
        eps = 1e-6 
        temperature_rewards = -np.abs(self.temperatures - self.ideal_temperatures) * np.log(np.abs(self.temperatures - self.ideal_temperatures + eps))
        temperature_rewards_sum = 0.25*np.sum(temperature_rewards)

        energy_reward = 1 if self.energy_level >= self.energy_threshold else -1

        rewards = temperature_rewards_sum + energy_reward

        # Check if the episode is done
        done = self.current_step >= self.time_steps

        # Increment the current time step
        self.current_step += 1

        # Add or remove people from the room every add_remove_frequency time steps
        if self.current_step % self.add_remove_frequency == 0:
            for i in range(self.num_bedrooms):
                if self.num_people_in_rooms[i] < self.max_people_per_room:
                    self.num_people_in_rooms[i] += 1
                    break

        # Exploration decay
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

        # Choose action with ε-greedy exploration
        if random.random() < self.epsilon:
            action = self.action_space.sample()

        # Additional info can be an empty dictionary
        info = {}

        # Calculate normalized observation
        normalized_observation = np.hstack(((self.temperatures - 16) / 17, (self.energy_level - 0) / self.energy_capacity))
        normalized_observation = normalized_observation.reshape((-1, 1))

        # Return the next state, reward, whether the episode is done, and additional info
        return normalized_observation, rewards, done, info

    def reset(self):
        # Reset the current time step
        self.current_step = 0
        
        # Check if it's time for home cleaning
        if self.current_step % self.cleaning_frequency == 0:
            # Consume energy for home cleaning
            self.energy_level -= self.cleaning_energy_consumption

            # Clip energy level within the valid range
            self.energy_level = max(0, min(self.energy_capacity, self.energy_level))

        # Normalize observation space
        normalized_observation = np.hstack(((self.temperatures - 16) / 17, (self.energy_level - 0) / self.energy_capacity))
        normalized_observation = normalized_observation.reshape((-1, 1))

        # Return the initial state
        return normalized_observation

# Test enviroment

In [193]:
# Custom Enviroment for energy saving
env_energy_saving = EnergySavingEnv()

In [194]:
env_energy_saving.observation_space.sample()

array([[ 0.41186544],
       [ 0.49269992],
       [ 0.10544687],
       [-0.05293326],
       [-0.46359992]], dtype=float32)

In [195]:
env_energy_saving.action_space.sample()

20

In [196]:
env_energy_saving.reset()

array([[0.54884314],
       [0.49603361],
       [0.52891721],
       [0.49428349],
       [0.476     ]])

In [197]:
episodes = 2
for episode in range(1, episodes+1):
  obs = env_energy_saving.reset()
  done = False
  score = 0

  while not done:
    # env_energy_saving.render()
    action = env_energy_saving.action_space.sample()
    obs, reward, done, info = env_energy_saving.step(action)
    score += reward

  print('Episode: {} Score {}'.format(episode, score))

Episode: 1 Score -485.9613365721448
Episode: 2 Score -240.86118006440438


# Train model

* PPO is a popular choice for environments with continuous actions and continuous observation spaces, like my resource management environment.
* For environments with continuous actions, I gonna use the MlpPolicy policy, which is a neural network policy with fully connected layers (multilayer perceptron).

In [198]:
log_path = os.path.join('Training', 'Logs')

In [199]:
energy_saving_model = DQN('MlpPolicy', env_energy_saving, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [200]:
energy_saving_model.learn(total_timesteps=500000)

Logging to Training/Logs/DQN_2
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 101      |
|    ep_rew_mean      | -635     |
|    exploration_rate | 0.992    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 3363     |
|    time_elapsed     | 0        |
|    total_timesteps  | 404      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 101      |
|    ep_rew_mean      | -684     |
|    exploration_rate | 0.985    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 3131     |
|    time_elapsed     | 0        |
|    total_timesteps  | 808      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 101      |
|    ep_rew_mean      | -618     |
|    exploration_rate | 0.977    |
| time/               | 

# Save model

In [201]:
path = os.path.join('Training', 'Saved Models', f'DQN_250k_energy_saving_model')
energy_saving_model.save(path)

In [202]:
del energy_saving_model

In [203]:
energy_model = DQN.load(path, env_energy_saving)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# Evaluate model

In [204]:
mean_reward, std_reward = evaluate_policy(energy_model, env_energy_saving, n_eval_episodes=10, render=True)

print(f"Mean reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Mean reward:47.81 +/- 0.00


In [205]:
episodes = 5

for episode in range(1, episodes+1):
    obs = env_energy_saving.reset() 
    done = False
    score = 0

    while not done:
        # obs = obs.reshape((1, -1))
        action, _ = energy_model.predict(obs)  
        obs, reward, done, info = env_energy_saving.step(action)
        score += reward

    print('Episode: {} Score {}'.format(episode, score))
    # env.close()  


Episode: 1 Score 36.63091540950615
Episode: 2 Score 40.789799242865755
Episode: 3 Score 46.77327531271288
Episode: 4 Score 43.261674642370394
Episode: 5 Score 40.39736088504285


# Viewing logs in Tensorboard

* **Average Reward:** Indicates how well the model perform in the particular enviroment.
* **Average Episode Length:** Indicates how long the agent lost in the particular enviroment.

In [208]:
training_log_path = os.path.join(log_path, 'DQN_2')

In [209]:
!tensorboard --logdir={training_log_path}

2024-05-01 18:49:45.938128: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-01 18:49:47.526351: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.16.2 at http://localhost:6006/ (Press CTRL+C to quit)
^C


# Consumer Model

NameError: name 'os' is not defined

In [ ]:
import numpy as np
import random
import gym
import gym.spaces as spaces

class EnergySavingEnv(gym.Env):
    def __init__(self):
        # Define default parameters
        self.num_bedrooms = 3
        self.num_people_per_bedroom = [1, 2, 1]
        self.num_people_in_living_room = 2
        self.solar_panel_efficiency = 0.2
        self.solar_panel_area = 100
        self.initialize_environment()

    def initialize_environment(self):
        # Define the action space
        self.action_space = spaces.Discrete(28)  # Total number of actions: 7 temperature adjustments * 4 airflow adjustments
        
        # Define the observation space with normalized ranges
        num_rooms = self.num_bedrooms + 1  # Include living room as well
        self.observation_space = spaces.Box(low=-1, high=1, shape=(num_rooms + 1, 1), dtype=np.float32)

        # Define the initial temperature (random between 24 and 26 degrees Celsius) for each room
        self.initial_temperatures = np.random.uniform(24, 26, size=(num_rooms,))

        # Set the current temperature to the initial temperature
        self.temperatures = self.initial_temperatures.copy()

        # Initialize energy level for the whole house
        self.energy_level = np.array([120])

        # Define the ideal temperature for each room
        self.ideal_temperatures = [22] * num_rooms  # Assuming ideal temperature of 22 for all rooms

        # Define the resource manager properties
        self.energy_capacity = 250
        self.energy_threshold = 50  # Energy threshold for energy-saving mode
        self.energy_usage_factor = 2  # Factor to control energy usage for temperature adjustment
        self.base_airflow = 0  # Base airflow without energy consumption
        self.max_airflow = 3  # Maximum airflow that can be achieved with energy consumption
        self.airflow_energy_consumption = 0.5  # Energy consumption per unit increase in airflow

        # Define the number of time steps for the resource manager
        self.time_steps = 100

        # Initialize the current time step
        self.current_step = 0

        self.cleaning_energy_consumption = 1  # Energy consumed during home cleaning

        # Define the frequency of home cleaning (once a day/episode)
        self.cleaning_frequency = 1

        # Define outside electricity cost
        self.base_electricity_cost = 1  # Base cost per unit of electricity

        # Initialize solar radiation parameters
        self.solar_radiation_variation = 100  # Maximum variation in solar radiation
        self.solar_radiation_mean = 700  # Mean solar radiation during the day
        self.randomness_factor = 0.1  # Factor to control randomness

        # Define the maximum number of people in each room
        self.max_people_per_room = 3
        self.num_people_in_rooms = self.num_people_per_bedroom.copy() + [0] * (self.num_bedrooms + 1)  # Initialize with specified people in bedrooms
        self.num_people_in_rooms[-1] = self.num_people_in_living_room  # Set the number of people in the living room

        # Define the frequency of adding/removing people (every 20 time steps)
        self.add_remove_frequency = 25

        # Exploration parameters
        self.epsilon = 0.1  # Exploration rate
        self.min_epsilon = 0.01  # Minimum exploration rate
        self.epsilon_decay = 0.99  # Exploration decay rate

    def set_house_config(self, num_bedrooms=3, num_people_per_bedroom=[1, 2, 1], num_people_in_living_room=2):
        self.num_bedrooms = num_bedrooms
        self.num_people_per_bedroom = num_people_per_bedroom
        self.num_people_in_living_room = num_people_in_living_room
        self.initialize_environment()

    def set_solar_panel_config(self, solar_panel_area=100, solar_panel_efficiency=0.2):
        self.solar_panel_area = solar_panel_area
        self.solar_panel_efficiency = solar_panel_efficiency

    def get_solar_radiation(self, time_of_day):
        # Simulate time-dependent solar radiation
        # For simplicity, we'll assume a pattern with variations based on time of day

        # Define solar radiation ranges for different times of the day
        if time_of_day < 6:  # Night
            solar_radiation = np.random.uniform(0, 100)
        elif time_of_day < 9:  # Early morning
            solar_radiation = np.random.uniform(500, 700)
        elif time_of_day < 15:  # Afternoon
            solar_radiation = np.random.uniform(700, 1000)
        elif time_of_day < 19:  # Evening
            solar_radiation = np.random.uniform(500, 700)
        else:  # Night
            solar_radiation = 0

        # Introduce randomness with occasional complete randomness
        if random.random() < self.randomness_factor:
            solar_radiation = np.random.uniform(self.solar_radiation_mean - self.solar_radiation_variation,
                                                 self.solar_radiation_mean + self.solar_radiation_variation)

        return solar_radiation

    def step(self, action):
        temperature_action = action // 4 + 1  # Temperature adjustment
        airflow_action = action % 4  # Airflow adjustment

        # Determine the temperature change based on the action
        temp_changes = np.zeros_like(self.temperatures)
        temp_change = 0
        if temperature_action == 1:  # Decrease temperature by 1 degree
            temp_change = -1
        elif temperature_action == 2:  # Keep temperature (no change)
            temp_change = 0
        elif temperature_action == 3:  # Increase temperature by 1 degree
            temp_change = 1
        elif temperature_action == 4:  # Decrease temperature by 2 degrees
            temp_change = -2
        elif temperature_action == 5:  # Increase temperature by 2 degrees
            temp_change = 2
        elif temperature_action == 6:  # Decrease temperature by 3 degrees
            temp_change = -3
        elif temperature_action == 7:  # Increase temperature by 3 degrees
            temp_change = 3

        # Apply the temperature change to all rooms
        temp_changes[:self.num_bedrooms] = temp_change

        # Clip temperature within the valid range
        self.temperatures += temp_changes
        self.temperatures = np.clip(self.temperatures, 16, 33)

        # Calculate energy produced by solar panels
        solar_radiation = self.get_solar_radiation(self.current_step / self.time_steps * 24)
        solar_energy = self.solar_panel_area * self.solar_panel_efficiency * solar_radiation

        # Calculate energy usage for temperature adjustment
        energy_usage = np.abs(temp_changes) * self.energy_usage_factor

        # Adjust airflow based on the action and number of people in the room
        airflow = self.base_airflow + airflow_action * (self.max_airflow / (self.action_space.n // 4 - 1))
        airflow_energy_usage = airflow * self.airflow_energy_consumption  # Initialize airflow energy usage
        
        # Adjust airflow energy consumption based on the number of people in each room
        for room_idx, num_people in enumerate(self.num_people_in_rooms):
            if num_people > 0:
                airflow_energy_usage += self.airflow_energy_consumption * num_people

        # Calculate net energy (energy usage - solar energy)
        self.energy_level = min(self.energy_level + solar_energy, self.energy_capacity)
        self.energy_level -= (np.sum(energy_usage) + airflow_energy_usage)

        # Clip energy level within the valid range
        self.energy_level = np.clip(self.energy_level, 0, self.energy_capacity)

        # Calculate reward based on energy-saving state and temperature
        # temperature_rewards = -0.5 * (np.abs(self.temperatures - self.ideal_temperatures) ** 2)
        eps = 1e-6 
        temperature_rewards = -np.abs(self.temperatures - self.ideal_temperatures) * np.log(np.abs(self.temperatures - self.ideal_temperatures + eps))
        temperature_rewards_sum = 0.25*np.sum(temperature_rewards)

        energy_reward = 1 if self.energy_level >= self.energy_threshold else -1

        rewards = temperature_rewards_sum + energy_reward

        # Check if the episode is done
        done = self.current_step >= self.time_steps

        # Increment the current time step
        self.current_step += 1

        # Add or remove people from the room every add_remove_frequency time steps
        if self.current_step % self.add_remove_frequency == 0:
            for i in range(self.num_bedrooms):
                if self.num_people_in_rooms[i] < self.max_people_per_room:
                    self.num_people_in_rooms[i] += 1
                    break

        # Exploration decay
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

        # Choose action with ε-greedy exploration
        if random.random() < self.epsilon:
            action = self.action_space.sample()

        # Additional info can be an empty dictionary
        info = {}

        # Calculate normalized observation
        normalized_observation = np.hstack(((self.temperatures - 16) / 17, (self.energy_level - 0) / self.energy_capacity))
        normalized_observation = normalized_observation.reshape((-1, 1))

        # Return the next state, reward, whether the episode is done, and additional info
        return normalized_observation, rewards, done, info

    def reset(self):
        # Reset the current time step
        self.current_step = 0
        
        # Check if it's time for home cleaning
        if self.current_step % self.cleaning_frequency == 0:
            # Consume energy for home cleaning
            self.energy_level -= self.cleaning_energy_consumption

            # Clip energy level within the valid range
            self.energy_level = max(0, min(self.energy_capacity, self.energy_level))

        # Normalize observation space
        normalized_observation = np.hstack(((self.temperatures - 16) / 17, (self.energy_level - 0) / self.energy_capacity))
        normalized_observation = normalized_observation.reshape((-1, 1))

        # Return the initial state
        return normalized_observation

In [ ]:
env_energy_saving = EnergySavingEnv()

env_energy_saving.set_house_config(num_bedrooms=3, num_people_per_bedroom=[1, 2, 1], num_people_in_living_room=2)
env_energy_saving.set_solar_panel_config(solar_panel_area=100, solar_panel_efficiency=0.2)

In [ ]:
log_path = os.path.join('Training', 'Consumer_Logs')

In [ ]:
energy_saving_model = DQN('MlpPolicy', env_energy_saving, verbose=1, tensorboard_log=log_path)

In [ ]:
energy_saving_model.learn(total_timesteps=500000)

In [ ]:
path = os.path.join('Training', 'Saved Models', f'DQN_250k__Consumer_energy_saving_model')
energy_saving_model.save(path)

In [ ]:
del energy_saving_model
energy_model = DQN.load(path, env_energy_saving)

In [ ]:
mean_reward, std_reward = evaluate_policy(energy_model, env_energy_saving, n_eval_episodes=10, render=True)

print(f"Mean reward:{mean_reward:.2f} +/- {std_reward:.2f}")

In [ ]:
episodes = 5

for episode in range(1, episodes+1):
    obs = env_energy_saving.reset() 
    done = False
    score = 0

    while not done:
        # obs = obs.reshape((1, -1))
        action, _ = energy_model.predict(obs)  
        obs, reward, done, info = env_energy_saving.step(action)
        score += reward

    print('Episode: {} Score {}'.format(episode, score))
    # env.close()  


In [ ]:
training_log_path = os.path.join(log_path, 'DQN_1')

In [ ]:
!tensorboard --logdir={training_log_path}